In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import math
import seaborn as sns
import os

# %config IPCompleter.greedy=True
%load_ext autoreload
%autoreload 2

sc.settings.verbosity = 0
sc.logging.print_header()
sns.set_context("paper")

/home/serhiy/Desktop/code/2_single_cell/dulac_kaplan2023_dev_regulation_snRNAseq_scATACseq_mouse_hbc04623/data/hotspot_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.9.2 anndata==0.8.0 umap==0.5.3 numpy==1.22.3 scipy==1.8.1 pandas==1.5.3 scikit-learn==1.2.1 statsmodels==0.13.5 pynndescent==0.5.8


In [2]:
# import local module containing misc code, helps keep notebooks clean from commonly used functions
import sys
# sys.path.insert(0, '/scratchfs/cherring/brain_maturation/analysis/submittal_final/')
import new_misc_code as nmc 

## **Pull all significant devDEGs from logTMMs data pickel**

In [3]:
# load log2 TMM pseudo-bulked data from limma-voom
# logTMMs.pkl was saved in 11__dev-DEGs_age-trend-fits_rate-of-change.ipynb
# logTMMs = nmc.load_obj("data/dev_deg_v6/logTMMs.pkl")
# logTMMs = nmc.load_obj("data/dev_deg_v4/logTMMs.pkl")
# all genes
# logTMMs = nmc.load_obj("data/dev_deg_v8/logTMMs.filtered100.pkl")
# intersection with mutant
logTMMs = nmc.load_obj("data/dev_deg_v9/logTMMs.overlap.pkl")

In [4]:
# index genes in logTMMs dataframes are devDEGs
sig_genes = {}
for traj_itr, df_itr in logTMMs.items():
    sig_genes[traj_itr] = df_itr.index.values
sig_genes.keys()

dict_keys(['e-A1', 'e-A5', 'e-C3', 'e-C4', 'e-C5', 'e-C6', 'e-H2', 'e-H6', 'e-H8', 'e-L3', 'e-L4', 'e-L5', 'e-M1', 'e-M3', 'e-M4', 'e-M5', 'e-M6', 'e-M7', 'e-M8', 'e-M9', 'e-N1', 'e-N11', 'e-N5', 'e-N9', 'e-P4', 'e-X1', 'i-B1', 'i-B10', 'i-B11', 'i-B13', 'i-B2', 'i-B3', 'i-B4', 'i-B5', 'i-B6', 'i-B7', 'i-B8', 'i-B9', 'i-C1', 'i-H1', 'i-H10', 'i-H11', 'i-H2', 'i-H4', 'i-H5', 'i-H6', 'i-H7', 'i-L1', 'i-L2', 'i-L3', 'i-L5', 'i-L6', 'i-L7', 'i-M1', 'i-M11', 'i-M2', 'i-M5', 'i-M6', 'i-M7', 'i-M8', 'i-M9', 'i-P1', 'i-P2', 'i-S1', 'i-S6', 'i-S7', 'i-S8', 'i-V1', 'i-V3', 'i-V6', 'i-X1', 'i-X10', 'i-X12', 'i-X13', 'i-X14', 'i-X2', 'i-X3', 'i-X4', 'i-X5', 'i-X6', 'i-X7', 'i-X8', 'i-X9'])

In [5]:
for iii, jjj in sig_genes.items():
    print( iii, jjj.shape)

e-A1 (4015,)
e-A5 (1590,)
e-C3 (3577,)
e-C4 (7657,)
e-C5 (2458,)
e-C6 (2005,)
e-H2 (2352,)
e-H6 (1718,)
e-H8 (1230,)
e-L3 (2261,)
e-L4 (1785,)
e-L5 (727,)
e-M1 (234,)
e-M3 (840,)
e-M4 (1016,)
e-M5 (1424,)
e-M6 (3602,)
e-M7 (631,)
e-M8 (1111,)
e-M9 (2725,)
e-N1 (2162,)
e-N11 (496,)
e-N5 (1344,)
e-N9 (1626,)
e-P4 (1854,)
e-X1 (2828,)
i-B1 (5253,)
i-B10 (1214,)
i-B11 (4280,)
i-B13 (1928,)
i-B2 (3881,)
i-B3 (4072,)
i-B4 (2731,)
i-B5 (3240,)
i-B6 (4776,)
i-B7 (1110,)
i-B8 (3126,)
i-B9 (941,)
i-C1 (4149,)
i-H1 (4218,)
i-H10 (2115,)
i-H11 (2881,)
i-H2 (582,)
i-H4 (3182,)
i-H5 (3414,)
i-H6 (5104,)
i-H7 (3809,)
i-L1 (3136,)
i-L2 (3271,)
i-L3 (3035,)
i-L5 (1516,)
i-L6 (3056,)
i-L7 (3418,)
i-M1 (3233,)
i-M11 (4356,)
i-M2 (4307,)
i-M5 (4644,)
i-M6 (4271,)
i-M7 (3077,)
i-M8 (446,)
i-M9 (432,)
i-P1 (8320,)
i-P2 (3917,)
i-S1 (982,)
i-S6 (1015,)
i-S7 (1476,)
i-S8 (1598,)
i-V1 (3086,)
i-V3 (8734,)
i-V6 (1136,)
i-X1 (4618,)
i-X10 (2757,)
i-X12 (883,)
i-X13 (2635,)
i-X14 (3441,)
i-X2 (3886,)
i-X3 (3100,)

## **Query GO terms for each set of devDEGs**

In [6]:
# read in adata, to use observational data for each batch
# adata_fl_nm = "data/2023-05-16_input/seurat.excitatory_select.h5ad"
#adata_fl_nm = "data/2023-05-16_input/seurat.inhibitory_select.h5ad"
adata_fl_nm = "data/2023-05-16_input/seurat.merged.h5ad"
adata = sc.read(adata_fl_nm)

In [ ]:
# https://docs.scvi-tools.org/en/0.6.8/tutorials/scanpy.html
# for the merged dataset we need to calculate highly variable genes for background in GO calculations
# need to run with 200G
# sc.pp.filter_genes( adata, min_counts=5, inplace=True)
# adata_original = adata.copy()
# sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
# sc.pp.log1p(adata)
# sc.pp.highly_variable_genes(adata, n_top_genes = 2000)    
# highly_variable_genes = adata.var["highly_variable"]
# adata = adata[:, highly_variable_genes]
# adata.raw = adata

### Get background genes (genes with >5 counts) for GO term query

In [7]:
# we may have an issue here
# make sure all genes have at least 5 counts, should have been already done
# print(sum(adata.raw.X.sum(0).A1>5)==adata.shape[1])
#back_ground = adata.var_names.values.astype(str).tolist()

False


### Scanpy .var_names_make_unique() formats gene duplicate names in way that is not recognized by GProfiler 

In [22]:
# check how many gene names are changed by scanpy var_names_make_unique()
# any of the filtered_feature_bc_matrix.h5 from our study can be used here
# this can also be skipped without much change to the GO analysis
# gene_file = "/dd_userdata/usrdat03/userdata/cherring/data/cellranger_outputs/brain_maturation_RNA/RL1777_2d_v3/outs/filtered_feature_bc_matrix.h5"
# test_adata = sc.read_10x_h5( gene_file)
# raw_names = test_adata.var_names.values
# test_adata.var_names_make_unique()
# unq_names = test_adata.var_names.values.tolist()
# current_names = adata.var_names.values.astype(str).tolist()
# len(current_names)

2000

In [9]:
#test_adata.var_names_make_unique()
#unq_names = test_adata.var_names.values.tolist()

In [13]:
# function to change scanpy corrected gene names back to generic name, this does not change the result very much
#def get_preunique_names( current_names, raw_names=raw_names, unq_names=unq_names):
#    raw_current_names = np.zeros_like( current_names)
#    for itr, gene_itr in enumerate( current_names):
#        if gene_itr not in raw_names:
#            raw_current_names[itr] = raw_names[unq_names.index(gene_itr)]
#        else:
#            raw_current_names[itr] = gene_itr
#    print( f"{len( current_names) - sum( nmc.member_test( raw_current_names, current_names))} gene name changes")
#    return( raw_current_names)

In [6]:
#back_ground_raw = get_preunique_names( current_names)
# back_ground_raw = current_names
back_ground_raw = nmc.load_obj("data/dev_deg_v7/background.pkl")
len(back_ground_raw)

2000

In [7]:
from gprofiler import GProfiler
def query_genes( genes, bk_grd, p_thresh=0.05):
    # Gprofiler does not like Y_RNA, times out
    if 'Y_RNA' in genes:
        genes.remove('Y_RNA')
    # base url sets version used in our study
    gp = GProfiler( return_dataframe=True, base_url="https://biit.cs.ut.ee/gprofiler_archive3/e102_eg49_p15/")
    query_df = gp.profile( organism='hsapiens', query=genes, user_threshold=p_thresh, background=bk_grd)
    ####################################################
    query_df = query_df[np.in1d( query_df['source'].values, ['GO:MF','GO:BP'])]
    ####################################################
    # if you'd like less general GOs
#     parent_mk = np.array( [len( ii)>=0 for ii in query_df['parents'].values])
    return( query_df) #.loc[parent_mk])

In [8]:
sig_genes.keys()

dict_keys(['e-A1', 'e-A5', 'e-C3', 'e-C4', 'e-C5', 'e-C6', 'e-H2', 'e-H6', 'e-H8', 'e-L3', 'e-L4', 'e-L5', 'e-M1', 'e-M3', 'e-M4', 'e-M5', 'e-M6', 'e-M7', 'e-M8', 'e-M9', 'e-N1', 'e-N11', 'e-N5', 'e-N9', 'e-P4', 'e-X1', 'i-B1', 'i-B10', 'i-B11', 'i-B13', 'i-B2', 'i-B3', 'i-B4', 'i-B5', 'i-B6', 'i-B7', 'i-B8', 'i-B9', 'i-C1', 'i-H1', 'i-H10', 'i-H11', 'i-H2', 'i-H4', 'i-H5', 'i-H6', 'i-H7', 'i-L1', 'i-L2', 'i-L3', 'i-L5', 'i-L6', 'i-L7', 'i-M1', 'i-M11', 'i-M2', 'i-M5', 'i-M6', 'i-M7', 'i-M8', 'i-M9', 'i-P1', 'i-P2', 'i-S1', 'i-S6', 'i-S7', 'i-S8', 'i-V1', 'i-V3', 'i-V6', 'i-X1', 'i-X10', 'i-X12', 'i-X13', 'i-X14', 'i-X2', 'i-X3', 'i-X4', 'i-X5', 'i-X6', 'i-X7', 'i-X8', 'i-X9'])

In [9]:
# takes time to compute
all_q_df = {}
for k_itr in sig_genes.keys():
    gene_itr = sig_genes[k_itr]
    # raw_gene_itr = get_preunique_names( gene_itr)
    raw_gene_itr = gene_itr
    q_df = query_genes( raw_gene_itr.tolist(), bk_grd=back_ground_raw, p_thresh=0.05)
    all_q_df[k_itr] = q_df

In [50]:
#all_q_df = nmc.load_obj("data/dev_deg_v6/devDEG_Gene-Ontology-hits.pkl")
#all_q_df = nmc.load_obj("data/dev_deg_v4/devDEG_Gene-Ontology-hits.pkl")

In [37]:
nmc.save_obj( all_q_df, "data/dev_deg_v9/devDEG_Gene-Ontology-hits.pkl")

In [38]:
len(all_q_df.keys())
gam_fits = nmc.load_obj("data/dev_deg_v8/AGE_gam_fits_12_grid100.filtered100.pkl")
len(gam_fits.keys())

140

In [39]:
all_q_df.keys()

dict_keys(['e-A1', 'e-A2', 'e-A3', 'e-A4', 'e-A5', 'e-B1', 'e-B2', 'e-C1', 'e-C2', 'e-C3', 'e-C4', 'e-C5', 'e-C6', 'e-C7', 'e-C8', 'e-C9', 'e-F1', 'e-H10', 'e-H2', 'e-H3', 'e-H4', 'e-H5', 'e-H6', 'e-H7', 'e-H8', 'e-H9', 'e-L2', 'e-L3', 'e-L4', 'e-L5', 'e-L6', 'e-L8', 'e-M1', 'e-M10', 'e-M2', 'e-M3', 'e-M4', 'e-M5', 'e-M6', 'e-M7', 'e-M8', 'e-M9', 'e-N1', 'e-N10', 'e-N11', 'e-N2', 'e-N3', 'e-N4', 'e-N5', 'e-N6', 'e-N7', 'e-N8', 'e-N9', 'e-P1', 'e-P2', 'e-P3', 'e-P4', 'e-T1', 'e-T2', 'e-X1', 'i-B1', 'i-B10', 'i-B11', 'i-B13', 'i-B2', 'i-B3', 'i-B4', 'i-B5', 'i-B6', 'i-B7', 'i-B8', 'i-B9', 'i-C1', 'i-C2', 'i-H1', 'i-H10', 'i-H11', 'i-H2', 'i-H3', 'i-H4', 'i-H5', 'i-H6', 'i-H7', 'i-H8', 'i-H9', 'i-L1', 'i-L2', 'i-L3', 'i-L4', 'i-L5', 'i-L6', 'i-L7', 'i-M1', 'i-M10', 'i-M11', 'i-M12', 'i-M2', 'i-M3', 'i-M4', 'i-M5', 'i-M6', 'i-M7', 'i-M8', 'i-M9', 'i-P1', 'i-P2', 'i-P3', 'i-P4', 'i-S1', 'i-S3', 'i-S4', 'i-S5', 'i-S6', 'i-S7', 'i-S8', 'i-V1', 'i-V2', 'i-V3', 'i-V4', 'i-V5', 'i-V6', 'i-V7', '

In [40]:
#for cell_type in all_q_df:
for cell_type in gam_fits.keys():   
    print(cell_type)
    df = all_q_df[cell_type]
    df["cell_type"] = cell_type
    col = df.pop("cell_type")
    df.insert(0, "cell_type", col)
    df.to_csv(f"data/dev_deg_v8/{cell_type}_GO.csv")
go_result = pd.concat(list(all_q_df.values()))
go_result.to_csv("data/dev_deg_v8/devdeg_go.csv")

e-A1
e-A2
e-A3
e-A4
e-A5
e-B1
e-B2
e-C1
e-C2
e-C3
e-C4
e-C5
e-C6
e-C7
e-C8
e-C9
e-F1
e-H10
e-H2
e-H3
e-H4
e-H5
e-H6
e-H7
e-H8
e-H9
e-L2
e-L3
e-L4
e-L5
e-L6
e-L8
e-M1
e-M10
e-M2
e-M3
e-M4
e-M5
e-M6
e-M7
e-M8
e-M9
e-N1
e-N10
e-N11
e-N2
e-N3
e-N4
e-N5
e-N6
e-N7
e-N8
e-N9
e-P1
e-P2
e-P3
e-P4
e-T1
e-T2
e-X1
i-B1
i-B10
i-B11
i-B13
i-B2
i-B3
i-B4
i-B5
i-B6
i-B7
i-B8
i-B9
i-C1
i-C2
i-H1
i-H10
i-H11
i-H2
i-H3
i-H4
i-H5
i-H6
i-H7
i-H8
i-H9
i-L1
i-L2
i-L3
i-L4
i-L5
i-L6
i-L7
i-M1
i-M10
i-M11
i-M12
i-M2
i-M3
i-M4
i-M5
i-M6
i-M7
i-M8
i-M9
i-P1
i-P2
i-P3
i-P4
i-S1
i-S3
i-S4
i-S5
i-S6
i-S7
i-S8
i-V1
i-V2
i-V3
i-V4
i-V5
i-V6
i-V7
i-V8
i-V9
i-X1
i-X10
i-X12
i-X13
i-X14
i-X15
i-X16
i-X17
i-X2
i-X3
i-X4
i-X5
i-X6
i-X7
i-X8
i-X9


In [41]:
gene_cluster_ids = pd.read_csv("data/dev_deg_v8/gene_cluster_ids.csv")
clusters = pd.unique(gene_cluster_ids["gene_trend"])
go_cluster_df = {}
for cluster in clusters:
    subset = gene_cluster_ids[gene_cluster_ids["gene_trend"] == cluster]
    print(cluster)
    genes = subset["gene_name"].tolist()
    q_df = query_genes(genes, bk_grd=back_ground_raw, p_thresh=0.05)
    go_cluster_df[cluster] = q_df
for cluster in go_cluster_df:
    df = go_cluster_df[cluster]
    df["cluster"] = cluster
    col = df.pop("cluster")
    df.insert(0, "cluster", col)
    df.to_csv(f"data/dev_deg_v8/bycluster/{cluster}_GO.csv")
go_result = pd.concat(list(go_cluster_df.values()))
go_result.to_csv("data/dev_deg_v8/bycluster/devdeg_go.csv")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [42]:
clusters = pd.unique(gene_cluster_ids["gene_trend"])
go_cluster_df = {}
for cluster in clusters:
    subset = gene_cluster_ids[gene_cluster_ids["gene_trend"] == cluster]
    cell_types = pd.unique(subset["cell_type"])
    for cell_type in cell_types:
        subset1 = subset[subset["cell_type"] == cell_type]
        genes = subset1["gene_name"].tolist()
        q_df = query_genes(genes, bk_grd=back_ground_raw, p_thresh=0.05)
        index = f"{cluster}_{cell_type}"
        go_cluster_df[index] = q_df
for index in go_cluster_df:
    df = go_cluster_df[index]
    df["cluster_celltype"] = index
    col = df.pop("cluster_celltype")
    df.insert(0, "cluster_celltype", col)
    df.to_csv(f"data/dev_deg_v8/by_cluster_celltype/{index}_GO.csv")
go_result = pd.concat(list(go_cluster_df.values()))
go_result.to_csv("data/dev_deg_v8/by_cluster_celltype/devdeg_go.csv")

KeyboardInterrupt: 